In [8]:
DATA_PATH = 'C:/Users/amityu/Gel_Sheet_Data/'
#movie = 'Control 050721'
#movie = 'CCA60'
movie ='140721'
MOVIE_PATH = DATA_PATH +  movie + '/'
GRAPH_PATH = 'C:/Users/amityu/Gel_Sheet_Graph/'
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from  skimage.filters import gaussian
from tqdm.notebook import trange, tqdm
import pandas as pd
from objects import movie_structure
from objects.movie_structure import gaussian_curvature, mean_curvature

from preprocessing import preprocessing
import importlib
import seaborn as sns

import json

# Loading a JSON file
def load_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Saving data to a JSON file
def save_json(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

In [27]:
movie = 'control'
DATA_PATH = 'C:/Users/amityu/Gel_Sheet_Data/'
MOVIE_PATH = DATA_PATH +  movie + '/'
GRAPH_PATH = 'C:/Users/amityu/Gel_Sheet_Graph/'
gel_data = load_json(DATA_PATH + 'global/%s.json'%movie)

In [40]:
#reload modoule if changed
importlib.reload(movie_structure)

<module 'objects.movie_structure' from 'C:\\Users\\amityu\\DataspellProjects\\gel_sheets\\objects\\movie_structure.py'>

In [41]:
gel = movie_structure.Movie.from_plate_and_height(gel_data)


In [26]:
import json

# Loading a JSON file
def load_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Saving data to a JSON file
def save_json(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

# Example usage
file_path = r'C:\Users\amityu\Gel_Sheet_Data\global\control.txt'
output_path = r'C:\Users\amityu\Gel_Sheet_Data\global\control.json'
# Loading a JSON file
loaded_data = {
    "data_path": r'C:\Users\amityu\Gel_Sheet_Data\Control 050721\\',
    "name":      "Control 05/07/21"
}

print(loaded_data)

# Modifying the data
#loaded_data['new_key'] = 'new_value'

# Saving the modified data to a JSON file
save_json(loaded_data, output_path)


{'data_path': 'C:\\Users\\amityu\\Gel_Sheet_Data\\Control 050721\\\\', 'name': 'Control 05/07/21'}


In [42]:
def plane_correlation(plane1, plane2):
    c = np.corrcoef(plane1.flatten(), plane2.flatten())[0,1]
    return c

correlation_list = []
for tp in tqdm(gel.tp_list):
    plate = tp.get_plate_plane()
    height = tp.get_height_plane()
    correlation_list.append(plane_correlation(plate, height))
plt.scatter(range(correlation_list), correlation_list)
plt.show()


IndexError: index 194 is out of bounds for axis 0 with size 186